<a href="https://colab.research.google.com/github/Alberto-002/IA_y_RN_1910136/blob/main/PIA_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Brisseida Esmeralda Gonzales Cabriales - 1974257

Jesus Roman Aguirre Hernandez - 2053240

Jair Eduardo Benavides Garcia - 2052989

Jesus Alberto Flores Garza - 1910136

Montserrat Sarahi Barba Botello - 2041504

Raul Alejandro Garza Elizondo - 2066224

CODIGO COMPLETO

In [ ]:
import cv2
import RPi.GPIO as GPIO
import time

# --- CONFIGURACIÓN GPIO ---
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# ---------------- MOTOR 1 (180°) ----------------
motor1_pins = [17, 18, 27, 22]
for pin in motor1_pins:
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, 0)

# ---------------- MOTOR 2 (90°) ----------------
motor2_pins = [5, 6, 13, 19]
for pin in motor2_pins:
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, 0)

# ---------------- BOTÓN ----------------
boton_pin = 23
GPIO.setup(boton_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)

# ---------------- SECUENCIA DE MOTOR ----------------
secuencia = [
    [1,0,0,1],
    [1,0,0,0],
    [1,1,0,0],
    [0,1,0,0],
    [0,1,1,0],
    [0,0,1,0],
    [0,0,1,1],
    [0,0,0,1]
]

# ---------------- FUNCIÓN DE GIRO ----------------
def girar_motor(pines, pasos, sentido=1, delay=0.002):
    secuencia_giro = secuencia if sentido == 1 else secuencia[::-1]
    for i in range(pasos):
        for paso in secuencia_giro:
            for pin, val in zip(pines, paso):
                GPIO.output(pin, val)
            time.sleep(delay)
    for pin in pines:
        GPIO.output(pin, 0)

# 512 pasos = 1 vuelta completa (360°)
pasos_180 = int(512 / 2)
pasos_90 = int(512 / 4)
pasos_120 = int(512 * 120 / 360)  # nuevo giro de 120° para motor 2

# ---------------- RECONOCIMIENTO FACIAL ----------------
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("modelo_rostro.yml")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cam = cv2.VideoCapture(0)

ID_AUTORIZADO = 1
print("✅ Sistema iniciado - Buscando rostro autorizado...")

# Variables para estabilidad de detección
detecciones_consecutivas = 0
DETECCIONES_REQUERIDAS = 5
UMBRAL_CONFIANZA = 60

try:
    while True:
        ret, frame = cam.read()
        if not ret:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            detecciones_consecutivas = 0
            print("Sin rostros detectados...")
            time.sleep(0.5)
            continue

        rostro_autorizado_detectado = False

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            if roi_gray.size == 0:
                continue

            id_pred, conf = recognizer.predict(roi_gray)

            if id_pred == ID_AUTORIZADO and conf < UMBRAL_CONFIANZA:
                detecciones_consecutivas += 1
                print(f"Rostro autorizado detectado ({detecciones_consecutivas}/{DETECCIONES_REQUERIDAS}) conf={conf:.2f}")
            else:
                detecciones_consecutivas = 0
                print(f"Rostro no autorizado (ID={id_pred}, conf={conf:.2f})")

            if detecciones_consecutivas >= DETECCIONES_REQUERIDAS:
                rostro_autorizado_detectado = True
                detecciones_consecutivas = 0
                break

        if rostro_autorizado_detectado:
            print("Rostro confirmado - Apertura de puerta...")

            # --- Apertura (giro invertido) ---
            girar_motor(motor1_pins, pasos_180, sentido=-1)
            girar_motor(motor2_pins, pasos_90, sentido=-1)

            print("Puerta abierta. Esperando botón para cerrar...")

            while GPIO.input(boton_pin):
                time.sleep(0.1)

            print("Cerrando puerta...")

            # --- Cierre (motor 2 gira 120° en sentido normal) ---
            girar_motor(motor2_pins, pasos_120, sentido=1)
            girar_motor(motor1_pins, pasos_180, sentido=1)

            print("Puerta cerrada. Buscando de nuevo rostro autorizado...\n")
            time.sleep(2)

        time.sleep(0.3)

except KeyboardInterrupt:
    print("\n Programa interrumpido por el usuario.")

finally:
    cam.release()
    GPIO.cleanup()
    print("GPIO liberado y sistema apagado correctamente.")